In [1]:
# Import Packages
import pandas as pd
import requests
from io import BytesIO
from urllib.request import urlopen
import glob
from zipfile import ZipFile
import os

In [3]:
# Retrieve all agency codes
url = 'https://api.usaspending.gov'
endpoint = '/api/v2/bulk_download/list_agencies/'

payload={
    "type": "award_agencies",
    "agency": 0
}
response = requests.post(f"{url}{endpoint}", json=payload)
agency_json = response.json()

agency_list = []

for i in range(len(agency_json['agencies']['cfo_agencies'])):
    agency_list.append(agency_json['agencies']['cfo_agencies'][i]['toptier_agency_id'])

agency_json['agencies']['cfo_agencies']

[{'name': 'Department of Agriculture',
  'toptier_agency_id': 14,
  'toptier_code': '012'},
 {'name': 'Department of Commerce',
  'toptier_agency_id': 15,
  'toptier_code': '013'},
 {'name': 'Department of Defense',
  'toptier_agency_id': 126,
  'toptier_code': '097'},
 {'name': 'Department of Education',
  'toptier_agency_id': 80,
  'toptier_code': '091'},
 {'name': 'Department of Energy',
  'toptier_agency_id': 78,
  'toptier_code': '089'},
 {'name': 'Department of Health and Human Services',
  'toptier_agency_id': 68,
  'toptier_code': '075'},
 {'name': 'Department of Homeland Security',
  'toptier_agency_id': 63,
  'toptier_code': '070'},
 {'name': 'Department of Housing and Urban Development',
  'toptier_agency_id': 76,
  'toptier_code': '086'},
 {'name': 'Department of Justice',
  'toptier_agency_id': 17,
  'toptier_code': '015'},
 {'name': 'Department of Labor',
  'toptier_agency_id': 18,
  'toptier_code': '1601'},
 {'name': 'Department of State',
  'toptier_agency_id': 21,
  't

In [18]:
# Iterate through all agency codes to retrieve and download all funding data
url = 'https://api.usaspending.gov'
endpoint = '/api/v2/bulk_download/list_monthly_files/'

award_types = ['assistance'] # ADD 'contracts' BACK

for x in agency_list:
    for y in award_types:
        payload= {
            "agency": x,
            "fiscal_year": 2022,
            "type": y
        }

        response = requests.post(f"{url}{endpoint}", json=payload)
        data = response.json()

        print('Retrieved', str(x), str(y), 'zip file url for download.')

        for i in data['monthly_files']:
            if i['fiscal_year'] == 2022:
                with urlopen(i['url']) as zipresp:
                    with ZipFile(BytesIO(zipresp.read())) as zfile:
                        zfile.extractall('/Users/jack/PycharmProjects/exploration_of_data/usa_spending_files')
                        print('Downloaded', str(x), str(y),'.csv files.')

Retrieved 14 assistance zip file url for download.
Downloaded 14 assistance .csv files.
Retrieved 15 assistance zip file url for download.
Downloaded 15 assistance .csv files.
Retrieved 126 assistance zip file url for download.
Downloaded 126 assistance .csv files.
Retrieved 80 assistance zip file url for download.
Downloaded 80 assistance .csv files.
Retrieved 78 assistance zip file url for download.
Downloaded 78 assistance .csv files.
Retrieved 68 assistance zip file url for download.
Downloaded 68 assistance .csv files.
Retrieved 63 assistance zip file url for download.
Downloaded 63 assistance .csv files.
Retrieved 76 assistance zip file url for download.
Downloaded 76 assistance .csv files.
Retrieved 17 assistance zip file url for download.
Downloaded 17 assistance .csv files.
Retrieved 18 assistance zip file url for download.
Downloaded 18 assistance .csv files.
Retrieved 21 assistance zip file url for download.
Downloaded 21 assistance .csv files.
Retrieved 16 assistance zip fi

In [19]:
path = '/Users/jack/PycharmProjects/exploration_of_data/usa_spending_files'
csv_files = glob.glob(os.path.join(path, "*.csv"))

li=[]

# loop over the list of csv files
for f in csv_files:
    df = pd.read_csv(f, index_col=None, header=0, dtype={'recipient_city_code': object}, low_memory=False)
    li.append(df)

complete_df = pd.concat(li, axis=0, ignore_index=True)

print('Files downloaded and concatenated', len(complete_df.index), 'rows returned.')

Files downloaded and concatenated 7317227 rows returned.


In [22]:
# Filter for only necessary columns
filtered_complete_df = complete_df[[
    'assistance_transaction_unique_key',
    'assistance_award_unique_key',
    'business_types_code',
    'federal_action_obligation',
    'total_obligated_amount',
    'award_id_fain',
    'action_date',
    'action_date_fiscal_year',
    'period_of_performance_start_date',
    'period_of_performance_current_end_date',
    'awarding_agency_name',
    'awarding_sub_agency_name',
    'funding_agency_name',
    'program_activities_funding_this_award',
    'recipient_uei',
    'recipient_duns',
    'recipient_name',
    'recipient_address_line_1',
    'recipient_city_name',
    'recipient_state_name',
    'recipient_zip_code',
    'recipient_parent_uei',
    'recipient_parent_duns',
    'recipient_parent_name',
    'cfda_number',
    'cfda_title',
    'funding_opportunity_number']]

filtered_complete_df = filtered_complete_df[filtered_complete_df['federal_action_obligation']>0]

print('Complete dataframe filtered', len(filtered_complete_df.index), 'rows returned. Ready for Salesforce join.')

Complete dataframe filtered 4852766 rows returned. Ready for Salesforce join.


In [6]:
# Remove files from folder
pattern = r"/Users/jack/PycharmProjects/exploration_of_data/usa_spending_files/*.csv"

for item in glob.iglob(pattern, recursive=True):
    # delete file
    os.remove(item)

print('Files removed from folder.')

Files removed from folder.


In [7]:
prime_award_df = filtered_complete_df[(filtered_complete_df['recipient_uei'].isna() == False) & (filtered_complete_df['recipient_state_name'].isna() == False)]

print('Award dataframe filtered for null Recipient UEI and State Name,', len(prime_award_df.index), 'rows returned.')

Award dataframe filtered for null Recipient UEI and name, 691051 rows returned.


In [29]:
client_df = pd.read_csv('/Users/Jack/Desktop/databricks_sfdc_accounts.csv')

# Clean prime award data for merge

prime_award_df['recipient_name'] = prime_award_df['recipient_name'].str.lower()
prime_award_df['recipient_parent_name'] = prime_award_df['recipient_parent_name'].str.lower()
prime_award_df['recipient_city_name'] = prime_award_df['recipient_city_name'].str.title()
prime_award_df['recipient_state_name'] = prime_award_df['recipient_state_name'].str.title()
#prime_award_df.loc[:,'total_obligated_amount'] = prime_award_df['total_obligated_amount'].map('{:,.0f}'.format)
prime_award_df['period_of_performance_start_date'] =  pd.to_datetime(prime_award_df['period_of_performance_start_date'], format='%Y-%m-%d', errors = 'coerce')
prime_award_df['period_of_performance_current_end_date'] =  pd.to_datetime(prime_award_df['period_of_performance_current_end_date'], format='%Y-%m-%d', errors = 'coerce')

def df_name_clean (df_name, col_name):
    df_name[col_name] = df_name[col_name]\
    .str.replace(',incorporated', '')\
    .str.replace('!', '')\
    .str.replace('.inc.', '')\
    .str.replace(',inc.', '')\
    .str.replace(', inc.', '')\
    .str.replace(' inc.', '')\
    .str.replace(' inc', '')\
    .str.replace(',inc', '')\
    .str.replace(', inc', '')\
    .str.replace(',, the', '')\
    .str.replace('charities archdiocese', 'charities of the archdiocese of')\
    .str.replace('charities diocese of', 'charities of the diocese of')\
    .str.replace('of of', 'of')\
    .str.replace(', the', '')\
    .str.rstrip(', ')

client_df = client_df.rename(columns={
    'Billing State/Province' : 'client_state',
    'Billing City' : 'client_city',
    'Unique ID' : 'id',
    'Account Name' : 'account_name',
    'Zip Code': 'client_zip_code'
})

client_df['name'] = client_df['name'].str.lower()
prime_award_df['recipient_zip_code'] = pd.to_numeric(prime_award_df['recipient_zip_code'], errors='coerce').fillna(0)
prime_award_df['recipient_zip_code'] = prime_award_df['recipient_zip_code'].astype(int)
prime_award_df['recipient_zip_code'] = prime_award_df['recipient_zip_code'].apply(lambda x: '{0:0>5}'.format(x))
client_df['zip_code_id'] = pd.to_numeric(client_df['zip_code_id'], errors='coerce').fillna(0)
client_df['zip_code_id'] = client_df['zip_code_id'].astype(int)
client_df['zip_code_id'] = client_df['zip_code_id'].apply(lambda x: '{0:0>5}'.format(x))

df_name_clean(client_df, 'name')
df_name_clean(prime_award_df, 'recipient_name')

/var/folders/s6/mh3v3q3d25s567j9vpd2g6xw0000gq/T/ipykernel_35591/1517147862.py:1: DtypeWarning: Columns (4,5,27,29,31,32,33,37,39,40,41,49,50,51,52,56,58,59,63,65,75,76,77,79,80,81,82,84,85,88,89,90,93,95,100,102,103,105,106,107,109,111,112,118,119,120,121,122,130,132,134,137,138,140,142,143,145,148,149,150,151,152,153,154,155,158,159,160,161,162,163,164,165,167,171,172,173) have mixed types. Specify dtype option on import or set low_memory=False.
  client_df = pd.read_csv('/Users/Jack/Desktop/databricks_sfdc_accounts.csv')
/var/folders/s6/mh3v3q3d25s567j9vpd2g6xw0000gq/T/ipykernel_35591/1517147862.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prime_award_df['recipient_name'] = prime_award_df['recipient_name'].str.lower()
/var/folders/s6/mh

In [33]:
# Join SFDC accounts onto USA Spending Award data

prime_award_merged_df = client_df.merge(
    prime_award_df,
    left_on = ['name', 'billing_state', 'billing_city'],#, 'zip_code_id'],
    right_on = ['recipient_name', 'recipient_state_name', 'recipient_city_name'],#, 'recipient_zip_code'],
    suffixes = ['_left', '_right'],
    how = 'inner'
)

print('Using Name, City, State, matching current clients onto USA spending produced ' + str(prime_award_merged_df.shape[0]) + ' matches')

print('USA Spending returns ' + str(prime_award_merged_df[['recipient_uei_right']].drop_duplicates().shape[0]) + ' unique accounts')

Using Name, City, State, and Zip Code, matching current clients onto USA spending produced 162610 matches
USA Spending returns 14481 unique accounts


In [127]:
client_mapping_dict = dict(zip(prime_award_merged_df.id, prime_award_merged_df.recipient_uei_right))
client_mapping_df = pd.DataFrame.from_dict(client_mapping_dict, orient='index').reset_index()

In [116]:
# Iterate through all agency codes to retrieve and download all funding data
url = 'https://api.usaspending.gov'
endpoint = '/api/v2/bulk_download/list_monthly_files/'

award_types = ['contracts']

for x in agency_list:
    for y in award_types:
        payload= {
            "agency": x,
            "fiscal_year": 2022,
            "type": y
        }

        response = requests.post(f"{url}{endpoint}", json=payload)
        data = response.json()

        print('Retrieved', str(x), str(y), 'zip file url for download.')

        for i in data['monthly_files']:
            if i['fiscal_year'] == 2022:
                with urlopen(i['url']) as zipresp:
                    with ZipFile(BytesIO(zipresp.read())) as zfile:
                        zfile.extractall('/Users/jack/PycharmProjects/exploration_of_data/usa_spending_files')
                        print('Downloaded', str(x), str(y),'.csv files.')

Retrieved 14 contracts zip file url for download.
Downloaded 14 contracts .csv files.
Retrieved 15 contracts zip file url for download.
Downloaded 15 contracts .csv files.
Retrieved 126 contracts zip file url for download.
Downloaded 126 contracts .csv files.
Retrieved 80 contracts zip file url for download.
Downloaded 80 contracts .csv files.
Retrieved 78 contracts zip file url for download.
Downloaded 78 contracts .csv files.
Retrieved 68 contracts zip file url for download.
Downloaded 68 contracts .csv files.
Retrieved 63 contracts zip file url for download.
Downloaded 63 contracts .csv files.
Retrieved 76 contracts zip file url for download.
Downloaded 76 contracts .csv files.
Retrieved 17 contracts zip file url for download.
Downloaded 17 contracts .csv files.
Retrieved 18 contracts zip file url for download.
Downloaded 18 contracts .csv files.
Retrieved 21 contracts zip file url for download.
Downloaded 21 contracts .csv files.
Retrieved 16 contracts zip file url for download.
Do

In [117]:
path = '/Users/jack/PycharmProjects/exploration_of_data/usa_spending_files'
csv_files = glob.glob(os.path.join(path, "*.csv"))

li=[]

# loop over the list of csv files
for f in csv_files:
    df = pd.read_csv(f, index_col=None, header=0, dtype={'recipient_city_code': object}, low_memory=False)
    li.append(df)

complete_df = pd.concat(li, axis=0, ignore_index=True)

print('Files downloaded and concatenated', len(complete_df.index), 'rows returned.')

Files downloaded and concatenated 4900159 rows returned.


In [134]:
contract_df = complete_df[(complete_df['federal_action_obligation']>0) & (filtered_complete_df['recipient_uei'].isna()==False)]

/var/folders/s6/mh3v3q3d25s567j9vpd2g6xw0000gq/T/ipykernel_35591/1929984851.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  contract_df = complete_df[(complete_df['federal_action_obligation']>0) & (filtered_complete_df['recipient_uei'].isna()==False)]


In [136]:
contract_df['recipient_name'] = contract_df['recipient_name'].str.lower()
contract_df['recipient_parent_name'] = contract_df['recipient_parent_name'].str.lower()
contract_df['recipient_city_name'] = contract_df['recipient_city_name'].str.title()
contract_df['recipient_state_name'] = contract_df['recipient_state_name'].str.title()
contract_df['period_of_performance_start_date'] =  pd.to_datetime(contract_df['period_of_performance_start_date'], format='%Y-%m-%d', errors = 'coerce')
contract_df['period_of_performance_current_end_date'] =  pd.to_datetime(contract_df['period_of_performance_current_end_date'], format='%Y-%m-%d', errors = 'coerce')

df_name_clean(contract_df, 'recipient_name')

/var/folders/s6/mh3v3q3d25s567j9vpd2g6xw0000gq/T/ipykernel_35591/2885479390.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contract_df['recipient_name'] = contract_df['recipient_name'].str.lower()
/var/folders/s6/mh3v3q3d25s567j9vpd2g6xw0000gq/T/ipykernel_35591/2885479390.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contract_df['recipient_parent_name'] = contract_df['recipient_parent_name'].str.lower()
/var/folders/s6/mh3v3q3d25s567j9vpd2g6xw0000gq/T/ipykernel_35591/2885479390.py:3: SettingWithC

In [137]:
# Join SFDC accounts onto USA Spending Award data

contract_merged = client_df.merge(
    contract_df,
    left_on = ['name', 'billing_state', 'billing_city'],
    right_on = ['recipient_name', 'recipient_state_name', 'recipient_city_name'],
    suffixes = ['_left', '_right'],
    how = 'inner'
)

In [140]:
print('Using Name, City, State, matching current clients onto USA spending produced ' + str(contract_merged.shape[0]) + ' matches')

print('USA Spending returns ' + str(contract_merged[['recipient_uei_left']].drop_duplicates().shape[0]) + ' unique accounts')
print('USA Spending returns ' + str(contract_merged[['recipient_uei_right']].drop_duplicates().shape[0]) + ' unique accounts')

Using Name, City, State, matching current clients onto USA spending produced 23579 matches
USA Spending returns 44 unique accounts
USA Spending returns 129 unique accounts


In [141]:
contract_merged['action_date'] = pd.to_datetime(contract_merged['action_date'])

contract_merged[(contract_merged['action_date'].dt.month == 9)]

,id,is_deleted,name,type,parent_id,recipient_uei_left,phone,fax,website,industry,...,highly_compensated_officer_2_name,highly_compensated_officer_2_amount,highly_compensated_officer_3_name,highly_compensated_officer_3_amount,highly_compensated_officer_4_name,highly_compensated_officer_4_amount,highly_compensated_officer_5_name,highly_compensated_officer_5_amount,usaspending_permalink,last_modified_date_right
63,0015b00001lHfIIAA0,False,booz allen hamilton,Prospect,NaN,NaN,(703) 902-5000,(703) 902-3333,www.bah.com,Non-Profit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_47Q...,2022-09-01 17:29:45
465,0015b00001lHfnFAAS,False,university of iowa,Prospect,NaN,Z1H9VJS8NG16,(319) 335-3500,(319) 356-3900,http://www.uiowa.edu,Public-Sector State,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_75N...,2022-09-01 13:17:52
2092,001j000000OKNMyAAP,False,national industries for the blind,Prospect,NaN,NaN,(703) 310-0500,(703) 310-0483,www.nib.org,Non-Profit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_47Q...,2022-09-02 11:46:52
2093,001j000000OKNMyAAP,False,national industries for the blind,Prospect,NaN,NaN,(703) 310-0500,(703) 310-0483,www.nib.org,Non-Profit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_47Q...,2022-09-02 13:49:23
2096,001j000000OKNMyAAP,False,national industries for the blind,Prospect,NaN,NaN,(703) 310-0500,(703) 310-0483,www.nib.org,Non-Profit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_47Q...,2022-09-02 11:58:12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21853,001j000000OKNMyAAP,False,national industries for the blind,Prospect,NaN,NaN,(703) 310-0500,(703) 310-0483,www.nib.org,Non-Profit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_47Q...,2022-09-04 17:29:12
21878,001j000000OKNMyAAP,False,national industries for the blind,Prospect,NaN,NaN,(703) 310-0500,(703) 310-0483,www.nib.org,Non-Profit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_47Q...,2022-09-02 12:46:22
22711,001j000000OKNMyAAP,False,national industries for the blind,Prospect,NaN,NaN,(703) 310-0500,(703) 310-0483,www.nib.org,Non-Profit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_47Q...,2022-09-04 15:33:37
22712,001j000000OKNMyAAP,False,national industries for the blind,Prospect,NaN,NaN,(703) 310-0500,(703) 310-0483,www.nib.org,Non-Profit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_47Q...,2022-09-04 17:05:59


In [147]:
contract_merged[(contract_merged['type'] == 'Customer')].to_csv('//Users/jack/Desktop/q3_qbr/contract.csv')